In [ ]:
import os

# replace with your local file path
image_dir = '/Users/cortesb/vip-amp-perception/AKS_cones/images'
mask_dir = '/Users/cortesb/vip-amp-perception/AKS_cones/masks'
cropped_dir = '/Users/cortesb/vip-amp-perception/AKS_cones/cropped_images'

image_files = os.listdir(image_dir)
mask_files = os.listdir(mask_dir)
cropped_files = os.listdir(cropped_dir)

print(len(image_files))
print(len(mask_files))
print(len(cropped_files))

3296
5150
5150


In [ ]:
image_mask_map = {}

for image_file in image_files:
    base_name = image_file.replace('.jpg', '')
    matching_masks = []
    for mask_file in mask_files:
        # Check if the mask file starts with the image base name
        if mask_file.startswith(base_name):
            matching_masks.append(mask_file)
    image_mask_map[image_file] = matching_masks

print(image_mask_map)

{'43a788f4d8e44a468b32fdc0ac12ff6f_jpg.rf.f4ccd5d32457874de62e4ba413fe7649.jpg': ['43a788f4d8e44a468b32fdc0ac12ff6f_jpg.rf.f4ccd5d32457874de62e4ba413fe7649_mask_class1_0.png'], 'mms_00114_jpg.rf.b2a767858c21c47d857e41993088673a.jpg': ['mms_00114_jpg.rf.b2a767858c21c47d857e41993088673a_mask_class1_0.png', 'mms_00114_jpg.rf.b2a767858c21c47d857e41993088673a_mask_class1_1.png'], 'amz_01677_png.rf.feca1df2bd71b50c0b0819cc956e539d.jpg': ['amz_01677_png.rf.feca1df2bd71b50c0b0819cc956e539d_mask_class1_0.png', 'amz_01677_png.rf.feca1df2bd71b50c0b0819cc956e539d_mask_class1_1.png'], 'cone-242-_jpg.rf.0a44008533074b26d170ad750b61c0fc.jpg': ['cone-242-_jpg.rf.0a44008533074b26d170ad750b61c0fc_mask_class1_0.png'], '98aadb3960f2434b9f9c351674dd9736_jpg.rf.ea9e2ebfc2c6f9c40f0a07e9017c38be.jpg': ['98aadb3960f2434b9f9c351674dd9736_jpg.rf.ea9e2ebfc2c6f9c40f0a07e9017c38be_mask_class1_0.png', '98aadb3960f2434b9f9c351674dd9736_jpg.rf.ea9e2ebfc2c6f9c40f0a07e9017c38be_mask_class1_1.png'], '792fc983ea5642fb9ba3

In [ ]:
from PIL import Image
import numpy as np
import os

# Create the directory to save cropped images if it doesn't exist
cropped_images_dir = '/content/drive/MyDrive/AMP/Synthetic_Data/AKS_cones/cropped_images'
os.makedirs(cropped_images_dir, exist_ok=True)

for image_file, matching_masks in image_mask_map.items():
    # Load the original image
    original_image_path = os.path.join(image_dir, image_file)
    original_image = Image.open(original_image_path).convert("RGB")

    for mask_file in matching_masks:
        # Load the mask
        mask_path = os.path.join(mask_dir, mask_file)
        mask_image = Image.open(mask_path).convert("L")  # Convert to grayscale

        # Find the bounding box of the mask (where pixel values are not zero)
        mask_np = np.array(mask_image)
        white_pixels = np.where(mask_np > 0) # Assuming non-zero pixels in mask indicate the object
        if white_pixels[0].size > 0:
            y_min, y_max = np.min(white_pixels[0]), np.max(white_pixels[0])
            x_min, x_max = np.min(white_pixels[1]), np.max(white_pixels[1])

            # Crop the original image using the bounding box
            cropped_image_bbox = original_image.crop((x_min, y_min, x_max + 1, y_max + 1))

            # Create a new image with an alpha channel
            cropped_image_exact = Image.new("RGBA", cropped_image_bbox.size)

            # Paste the cropped image using the mask as the alpha channel
            mask_cropped = mask_image.crop((x_min, y_min, x_max + 1, y_max + 1))
            cropped_image_exact.paste(cropped_image_bbox, (0, 0), mask_cropped)

            # Save the cropped image
            save_path = os.path.join(cropped_images_dir, mask_file)
            cropped_image_exact.save(save_path)

            print(f"Saved cropped image for mask: {mask_file}")
        else:
            print(f"No non-zero pixels found in mask: {mask_file} for image {image_file}")


Streaming output truncated to the last 5000 lines.
Saved cropped image for mask: BME_01283_jpg.rf.89f78b5fce9e2e10a3bbf1364b8bac61_mask_class1_0.png
Saved cropped image for mask: BME_01283_jpg.rf.89f78b5fce9e2e10a3bbf1364b8bac61_mask_class1_1.png
Saved cropped image for mask: eco_00245_png.rf.de387af038dabb872a933f27f8ea0c02_mask_class1_0.png
Saved cropped image for mask: eco_00245_png.rf.de387af038dabb872a933f27f8ea0c02_mask_class1_1.png
Saved cropped image for mask: 2a2564f96e884d8bb37d00592b0c18c1_jpg.rf.22b7293827ebb5e1918baf6387c4902c_mask_class1_0.png
Saved cropped image for mask: 64f3f054edd84d4798c54a7177ecd4b0_jpg.rf.be2ec4ac02746389aeb5dc82411d4561_mask_class1_1.png
Saved cropped image for mask: 64f3f054edd84d4798c54a7177ecd4b0_jpg.rf.be2ec4ac02746389aeb5dc82411d4561_mask_class1_2.png
Saved cropped image for mask: 33_jpeg.rf.26a9b2f0e15c2d4fd791df79e1f764ab_mask_class1_0.png
Saved cropped image for mask: 68_jpeg.rf.ec158e8b8f17a0d270a68881b5d582f9_mask_class1_0.png
Saved crop